# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,11.65,82,0,4.55,AU,1679859247
1,1,lompoc,34.6391,-120.4579,17.79,60,0,6.69,US,1679859248
2,2,zhigansk,66.7697,123.3711,-8.54,94,100,11.50,RU,1679859248
3,3,tasiilaq,65.6145,-37.6368,-10.91,81,19,1.51,GL,1679859248
4,4,lagoa,39.0500,-27.9833,13.87,86,100,11.74,PT,1679859249


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
%%capture --no-display

# Configure the map plot
city_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
city_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [31]:
# Narrow down cities that fit criteria and drop any results with null values
city_narrow = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & 
                               (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_narrow.dropna()

# Display sample data
city_narrow

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
121,121,arlington,32.7357,-97.1081,26.59,36,0,2.57,US,1679859137
169,169,sann,26.0418,68.1378,21.95,35,0,1.20,PK,1679859309
183,183,ures,29.4333,-110.4000,24.53,11,0,3.19,MX,1679859316
225,225,umm lajj,25.0213,37.2685,21.40,66,0,4.19,SA,1679859334
335,335,bilma,18.6853,12.9164,24.86,16,0,2.96,NE,1679859373
394,394,rivadavia,-33.1905,-68.4608,26.21,32,0,3.13,AR,1679859396
405,405,agadez,19.7500,10.2500,21.70,16,0,2.34,NE,1679859399
420,420,makakilo city,21.3469,-158.0858,26.18,64,0,4.12,US,1679859404
441,441,tigre,-34.4260,-58.5796,24.58,40,0,2.57,AR,1679859412
458,458,adrar,20.5022,-10.0711,26.07,6,0,3.93,MR,1679859418


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_narrow.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
121,arlington,US,32.7357,-97.1081,36,
169,sann,PK,26.0418,68.1378,35,
183,ures,MX,29.4333,-110.4000,11,
225,umm lajj,SA,25.0213,37.2685,66,
335,bilma,NE,18.6853,12.9164,16,
394,rivadavia,AR,-33.1905,-68.4608,32,
405,agadez,NE,19.7500,10.2500,16,
420,makakilo city,US,21.3469,-158.0858,64,
441,tigre,AR,-34.4260,-58.5796,40,
458,adrar,MR,20.5022,-10.0711,6,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
arlington - nearest hotel: Parkway Inn
sann - nearest hotel: No hotel found
ures - nearest hotel: Hacienda Quinta Nápoles
umm lajj - nearest hotel: No hotel found
bilma - nearest hotel: No hotel found
rivadavia - nearest hotel: Hotel Pasambay
agadez - nearest hotel: No hotel found
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
tigre - nearest hotel: Posada de 1860
adrar - nearest hotel: No hotel found
dubai - nearest hotel: Holiday Inn Bur Dubai - Embassy District
san carlos de bariloche - nearest hotel: Kronos


,City,Country,Lat,Lng,Humidity,Hotel Name
121,arlington,US,32.7357,-97.1081,36,Parkway Inn
169,sann,PK,26.0418,68.1378,35,No hotel found
183,ures,MX,29.4333,-110.4000,11,Hacienda Quinta Nápoles
225,umm lajj,SA,25.0213,37.2685,66,No hotel found
335,bilma,NE,18.6853,12.9164,16,No hotel found
394,rivadavia,AR,-33.1905,-68.4608,32,Hotel Pasambay
405,agadez,NE,19.7500,10.2500,16,No hotel found
420,makakilo city,US,21.3469,-158.0858,64,Embassy Suites by Hilton Oahu Kapolei
441,tigre,AR,-34.4260,-58.5796,40,Posada de 1860
458,adrar,MR,20.5022,-10.0711,6,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)